In [1]:
from scipy import optimize
import scipy.stats as st
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import math
import xarray as xr
import sys
import scipy.io
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt
import os.path
from os import path
import netCDF4
from statsmodels.tsa.stattools import acf
import netCDF4 as nc
import warnings
from scipy.spatial import cKDTree
from sklearn.datasets import make_blobs 
from scipy.stats import norm
import numpy as np
from scipy.spatial.distance import pdist, squareform
from scipy.integrate import trapz



warnings.filterwarnings("ignore")
cmap = plt.cm.RdBu

In [2]:
def morani(p):
    # Replace this with your actual 2D data of shape (192, 288)

    # Remove NaN values
    p_clean = p[~np.isnan(p).any(axis=1)]

    # Calculate mean of the cleaned data
    mean_p = np.mean(p_clean)

    # Number of data points
    n = p_clean.shape[0]

    # Calculate pairwise distances between data points
    distances = squareform(pdist(p_clean))

    # Define a threshold distance for spatial weights
    threshold = 1.0  # Adjust this based on your data characteristics

    # Create binary spatial weights matrix based on distance threshold
    w = np.where(distances <= threshold, 1, 0)

    # Calculate sums for Moran's I formula
    s1 = np.sum(w)
    s2 = np.dot(w, p_clean)
    s3 = p_clean - mean_p

    # Calculate Moran's I
    moran_i = (n / s1) * (np.dot(s2.T, s3) / np.dot(s3.T, s3))

    return moran_i


def gini(x):
    total = 0
    for i, xi in enumerate(x[:-1], 1):
        total += np.sum(np.abs(xi - x[i:]))
    return total / (len(x)**2 * np.mean(x))

def weighted_percentile(cli_pr, weights, percentile):
    # Flatten the 2D arrays and remove NaN values
    flat_array = cli_pr.flatten()
    valid_indices = np.logical_not(np.isnan(flat_array))
    sorted_indices = np.argsort(flat_array[valid_indices])
    sorted_array = flat_array[valid_indices][sorted_indices]
    sorted_weights = weights.flatten()[valid_indices][sorted_indices]
    
    # Calculate the cumulative sum of the sorted weights
    cumulative_weights = np.cumsum(sorted_weights)
    total_weight = cumulative_weights[-1]
    
    # Find the index corresponding to the desired percentile
    target_index = np.searchsorted(cumulative_weights, total_weight * (percentile / 100.0))
    
    # Get the value at the target index in the sorted array
    percentile_value = sorted_array[target_index]
    
    return percentile_value


def giniw(x, w=None):
# from https://stackoverflow.com/questions/48999542/more-efficient-weighted-gini-coefficient-in-python
    # The rest of the code requires numpy arrays.
#     x = np.asarray(x)
    x=x.flatten()
    w=w.flatten()
    if w is not None:
#         w = np.asarray(w)
        sorted_indices = np.argsort(x)
        sorted_x = x[sorted_indices]
        sorted_w = w[sorted_indices]
        # Force float dtype to avoid overflows
        cumw = np.cumsum(sorted_w)
        cumxw = np.cumsum(sorted_x * sorted_w)
        return (np.sum(cumxw[1:] * cumw[:-1] - cumxw[:-1] * cumw[1:]) /
                (cumxw[-1] * cumw[-1]))
    else:
#         x = np.asarray(x)
        sorted_x = np.sort(x)
        n = x.size
        cumx = np.cumsum(sorted_x)
        # The above formula, with all weights equal to 1 simplifies to:
        return (n + 1 - 2 * np.sum(cumx) / cumx[-1]) / n
    
def find_weighted_percentile_rank(cli_pr, weights, value):
    flat_array = cli_pr.flatten()
    flat_weights = weights.flatten()
    
    # Sort the values and corresponding weights
    sorted_indices = np.argsort(flat_array)
    sorted_array = flat_array[sorted_indices]
    sorted_weights = flat_weights[sorted_indices]
    
    # Calculate the cumulative sum of the sorted weights
    cumulative_weights = np.cumsum(sorted_weights)
    total_weight = cumulative_weights[-1]
    
    # Find the index of the value in the sorted array
    index = np.searchsorted(sorted_array, value)
    
    # Calculate the percentile rank considering weight
    percentile_rank = (cumulative_weights[index] / total_weight) * 100
    
    return percentile_rank

def calculate_mann_kendall(x):
    n = len(x)
    s = 0
    for k in range(n - 1):
        for j in range(k + 1, n):
            s += np.sign(x[j] - x[k])
    trend = s / (n * (n - 1) / 2)
    var_s = (n * (n - 1) * (2 * n + 5)) / 18
    if s > 0:
        z = (s - 1) / np.sqrt(var_s)
    elif s < 0:
        z = (s + 1) / np.sqrt(var_s)
    else:
        z = 0
    p = 2 * (1 - norm.cdf(abs(z)))
    return trend, p

def spa_lag(X, k):

    tree = cKDTree(X)
    spatial_lag = []

    for point in X:
        # Query the k nearest neighbors' indices
        _, indices = tree.query(point, k=k+1)  # +1 to exclude the point itself

        # Calculate the spatial lag by averaging the values of the k nearest neighbors
        spatial_lag_value = np.mean(X[indices[1:], :], axis=0)  # Exclude the first index which is the point itself
        spatial_lag.append(spatial_lag_value)

    spatial_lag = np.array(spatial_lag)
    return spatial_lag

import numpy as np
from scipy.spatial import cKDTree

def spa_lag_with_weights(X, W, k):
    tree = cKDTree(X)
    spatial_lag = []

    for i, point in enumerate(X):
        # Query the k nearest neighbors' indices
        _, indices = tree.query(point, k=k+1)  # +1 to exclude the point itself

        # Get the weights corresponding to the k nearest neighbors
        neighbor_weights = W[indices[1:]]

        # Normalize the weights
        normalized_weights = neighbor_weights / np.sum(neighbor_weights)

        # Calculate the spatial lag using weighted average without np.dot
        spatial_lag_value = np.sum(normalized_weights * X[indices[1:], :], axis=0)
        spatial_lag.append(spatial_lag_value)

    spatial_lag = np.array(spatial_lag)
    return spatial_lag


# Example usage
# X is your data matrix
# W is your spatial weight matrix
# k is the number of nearest neighbors to consider
# result = spa_lag_with_weights(X, W, k)

import numpy as np
from copy import copy
def calculate_local_moran(p, w):
    p = p.flatten()
    w = w.flatten()
    n = p.shape[0]
    mean_p = np.mean(p)
    variance_p = np.var(p)
    
    local_morans_i_values = np.zeros(n)
    
    for i in range(n):
        num = n * (p[i] - mean_p) * np.sum(w[i] * (p - mean_p))
        den = variance_p
        local_morans_i_values[i] = num / den
        
    return local_morans_i_values

# Example data and weights (replace with your actual data and weights)



def sgini(pts, w):
    w=w/np.nansum(w)
    sorted_indices = np.argsort(pts.flatten())
    sorted_x = pts.flatten()[sorted_indices]
    sorted_w = w.flatten()[sorted_indices]
    # Force float dtype to avoid overflows
    cumw = np.cumsum(sorted_w)-sorted_w/2
    cumx = np.cumsum(sorted_x)
    cumx=cumx/cumx[-1]
    x_clipped = np.clip(cumw/cumw[-1], 0, 1)
    area = trapz(cumx[x_clipped <= 1], x_clipped[x_clipped <= 1])   
    GS=(0.5-area)*2    
    return GS

def sgini_la(pts, la, w):
    w=w/np.nansum(w)

    sorted_indicesr = np.argsort(la.flatten())

    sorted_indices = np.argsort(pts.flatten())
    sorted_x = pts.flatten()[sorted_indicesr]
    sorted_w = w.flatten()[sorted_indices]
    # Force float dtype to avoid overflows
    cumw = np.cumsum(sorted_w)-sorted_w/2
    cumx = np.cumsum(sorted_x)
    cumx=cumx/cumx[-1]
    x_clipped = np.clip(cumw/cumw[-1], 0, 1)
    area = trapz(cumx[x_clipped <= 1], x_clipped[x_clipped <= 1]) 
    GS=(0.5-area)*2
    return GS





def random_rearrange_2d_array(p):
    # Flatten the 2D array into a 1D array
    flattened_p = p.flatten()
    
    # Shuffle the elements of the 1D array randomly
    np.random.shuffle(flattened_p)
    
    # Reshape the shuffled 1D array back into the original 2D array shape
    reshaped_p = flattened_p.reshape(p.shape)
    
    return reshaped_p


In [3]:
import xarray as xr
import glob
MODELINFO=[["ACCESS-ESM1-5",101,249],\
           ["CanESM5",1850,2000],\
           ["CESM2",1,150],\
           ["EC-Earth3-CC",1850,2000],\
           ["FGOALS-g3",370,525],\
           ["MIROC6",3200,3349],\
           ["MRI-ESM2-0",1850,2000],\
           ["NorESM2-LM",1,150],\
           ["NorESM2-MM",1,150],\
           ["GFDL-ESM4",1,150],\
           ["GFDL-CM4",1,150]]

GRIDINFO=[["/pr_day_ACCESS-ESM1-5_1pctCO2_r1i1p1f1_gn_01510101-02001231.nc"],\
          ["/pr_day_CanESM5_1pctCO2_r1i1p1f1_gn_18500101-20001231.nc"],\
          ["/pr_day_CESM2_1pctCO2_r1i1p1f1_gn_00710101-00801231.nc"],\
          ["/pr_day_EC-Earth3-CC_1pctCO2_r1i1p1f1_gr_19080101-19081231.nc"],\
          ["/pr_day_FGOALS-g3_1pctCO2_r1i1p1f1_gn_05160101-05161231.nc"],\
          ["/pr_day_MIROC6_1pctCO2_r1i1p1f1_gn_32800101-32891231.nc"],\
          ["/pr_day_MRI-ESM2-0_1pctCO2_r1i1p1f1_gn_18500101-18991231.nc"],\
          ["/pr_day_NorESM2-LM_1pctCO2_r1i1p1f1_gn_01210101-01301231.nc"],\
          ["/pr_day_NorESM2-MM_1pctCO2_r1i1p1f1_gn_01210101-01301231.nc"],\
          ["/pr_day_GFDL-ESM4_1pctCO2_r1i1p1f1_gr1_00610101-00801231.nc"],\
          ["/pr_day_GFDL-CM4_1pctCO2_r1i1p1f1_gr1_00210101-00401231.nc"]]

MASKINFO=[["/mrsos_day_ACCESS-ESM1-5_1pctCO2_r1i1p1f1_gn_01010101-01501231.nc"],\
          ["/mrsos_Lmon_CanESM5_1pctCO2_r1i1p1f1_gn_185001-200012.nc"],\
          ["/mrsos_Lmon_CESM2_1pctCO2_r1i1p1f1_gn_000101-005012.nc"],\
          ["/mrsos_Lmon_EC-Earth3-CC_1pctCO2_r1i1p1f1_gr_199601-199612.nc"],\
          ["/mrsos_Lmon_FGOALS-g3_1pctCO2_r1i1p1f1_gn_037301-052812.nc"],\
          ["/mrsos_Lmon_MIROC6_1pctCO2_r1i1p1f1_gn_320001-329912.nc"],\
          ["/mrsos_Lmon_MRI-ESM2-0_1pctCO2_r1i1p1f1_gn_185001-200012.nc"],\
          ["/mrsos_Lmon_NorESM2-LM_1pctCO2_r1i1p1f1_gn_007101-008012.nc"],\
          ["/mrsos_Lmon_NorESM2-MM_1pctCO2_r1i1p1f1_gn_007101-008012.nc"],\
          ["/mrsos_Lmon_GFDL-ESM4_1pctCO2_r1i1p1f1_gr1_010101-015012.nc"],\
          ["/mrsos_Lmon_GFDL-CM4_1pctCO2_r1i1p1f1_gr1_000101-010012.nc"]]

model_names = [model[0] for model in MODELINFO]
model_start = [model[1] for model in MODELINFO]
model_end = [model[2] for model in MODELINFO]
GRID = [model[0] for model in GRIDINFO]
MASK = [model[0] for model in MASKINFO]
lenyear=100
miss_val = -9.99e08


In [5]:
latuse = 30

for model in range(0,10):
    GINIland=np.array([np.full((lenyear),miss_val)])
    GINIsland=np.array([np.full((lenyear),miss_val)])

    GINIocean=np.array([np.full((lenyear),miss_val)])
    GINIsocean=np.array([np.full((lenyear),miss_val)])

    GINIglob=np.array([np.full((lenyear),miss_val)])
    GINIsglob=np.array([np.full((lenyear),miss_val)])
  
    MoranI=np.array([np.full((lenyear),miss_val)])
    
    mat1 = nc.Dataset('/scratch/cimes/hh9736/CMIP6_pr/' + model_names[model] + GRID[model])
    data1 = nc.Dataset('/scratch/cimes/hh9736/CMIP6_pr/' + model_names[model] + MASK[model])
    data2 = xr.open_dataset('/scratch/cimes/hh9736/CMIP6_pr/' + model_names[model] + MASK[model])
    lats = data2['lat'].values
    lats = lats.astype(int)
    expolar = np.logical_or(lats > latuse, lats < -latuse)
    
    kn=round(lats.shape[0]/18)
    
    latmax=np.size(mat1['pr'][1,:,1])+1
    lonmax=np.size(mat1['pr'][1,1,:])+1
    lenlat=latmax-1
    lenlon=lonmax-1
    lenyear=2100-1850+1
    miss_val = -9.99e08

    lats = data2['lat'].values
    lons = data2['lon'].values
    weights = np.cos(np.deg2rad(lats))
    weights /= np.sum(weights)
    weights_reshaped = np.reshape(weights, (1, len(lats), 1))
    weights_reshaped = np.repeat(weights_reshaped, len(lons), axis=2)


# Specify the path to your NetCDF files
    file_pattern = "/scratch/cimes/hh9736/CMIP6_pr/"+ model_names[model] +"/pr*_r1i1p1f1_*.nc"

    # Get a list of file paths matching the pattern
    file_paths = glob.glob(file_pattern)

    # Open and concatenate the files one by one
    datasets = []
    for file_path in file_paths:
        dataset = xr.open_dataset(file_path)
        datasets.append(dataset)

    # Concatenate the datasets along the time dimension
    data = xr.concat(datasets, dim="time")
    data = data.sortby('time')
    annual_mean = data.groupby('time.year').mean(dim='time')

    # Select the first 100 years of data
    start_year = annual_mean.year[0]
    end_year = start_year + 99
    YEARP = annual_mean.sel(year=slice(start_year, end_year))
    mask=np.array(data1['mrsos'])
    
    mask=mask[0,:,:]
    mask[mask>10000]=1.e+20
    mask[mask<0.000000000000000000000001]=1.e+20
   
    weights_reshaped=weights_reshaped[0,:,:]
    
    mat1=YEARP['pr']
    for yy in range(0,100):
        pts=np.array(mat1[yy,:,:])
  
        la=spa_lag(pts, kn)
        pts[expolar, :] = np.nan
        
        pts_land=copy(pts)
        pts_land[mask>10000]=np.nan
        GINIland[0,yy]=sgini(pts_land[~np.isnan(pts_land)], weights_reshaped[~np.isnan(pts_land)])
        GINIsland[0,yy]=sgini_la(pts_land[~np.isnan(pts_land)],la[~np.isnan(pts_land)], weights_reshaped[~np.isnan(pts_land)])
        
        pts_ocean=copy(pts)
        pts_ocean[mask<10000]=np.nan
        GINIocean[0,yy]=sgini(pts_ocean[~np.isnan(pts_ocean)], weights_reshaped[~np.isnan(pts_ocean)])
        GINIsocean[0,yy]=sgini_la(pts_ocean[~np.isnan(pts_ocean)],la[~np.isnan(pts_ocean)], weights_reshaped[~np.isnan(pts_ocean)])
        
        pts_glob=copy(pts)
        GINIglob[0,yy]=sgini(pts_glob[~np.isnan(pts_glob)], weights_reshaped[~np.isnan(pts_glob)])
        GINIsglob[0,yy]=sgini_la(pts_glob[~np.isnan(pts_glob)],la[~np.isnan(pts_glob)], weights_reshaped[~np.isnan(pts_glob)])
        
        
    np.save('/scratch/cimes/hh9736/CMIP6_pr/GINI_spa_'+ str(latuse) +'landP_' + model_names[model] + '.npy', np.array([GINIland]))
    np.save('/scratch/cimes/hh9736/CMIP6_pr/GINIs_spa_'+ str(latuse) +'landP_' + model_names[model] + '.npy', np.array([GINIsland]))
    
    np.save('/scratch/cimes/hh9736/CMIP6_pr/GINI_spa_'+ str(latuse) +'oceanP_' + model_names[model] + '.npy', np.array([GINIocean]))
    np.save('/scratch/cimes/hh9736/CMIP6_pr/GINIs_spa_'+ str(latuse) +'oceanP_' + model_names[model] + '.npy', np.array([GINIsocean]))
    
    np.save('/scratch/cimes/hh9736/CMIP6_pr/GINI_spa_'+ str(latuse) +'globP_' + model_names[model] + '.npy', np.array([GINIglob]))
    np.save('/scratch/cimes/hh9736/CMIP6_pr/GINIs_spa_'+ str(latuse) +'globP_' + model_names[model] + '.npy', np.array([GINIsglob]))


In [ ]:
latuse = 90
latnouse = 30

for model in range(0,11):
    GINIland=np.array([np.full((lenyear),miss_val)])
    GINIsland=np.array([np.full((lenyear),miss_val)])

    GINIocean=np.array([np.full((lenyear),miss_val)])
    GINIsocean=np.array([np.full((lenyear),miss_val)])

    GINIglob=np.array([np.full((lenyear),miss_val)])
    GINIsglob=np.array([np.full((lenyear),miss_val)])
  
    MoranI=np.array([np.full((lenyear),miss_val)])
    
    mat1 = nc.Dataset('/scratch/cimes/hh9736/CMIP6_pr/' + model_names[model] + GRID[model])
    data1 = nc.Dataset('/scratch/cimes/hh9736/CMIP6_pr/' + model_names[model] + MASK[model])
    data2 = xr.open_dataset('/scratch/cimes/hh9736/CMIP6_pr/' + model_names[model] + MASK[model])
    lats = data2['lat'].values
    lats = lats.astype(int)
    expolar = np.logical_and(lats > -latnouse, lats < latnouse)
    
    kn=round(lats.shape[0]/18)
    
    latmax=np.size(mat1['pr'][1,:,1])+1
    lonmax=np.size(mat1['pr'][1,1,:])+1
    lenlat=latmax-1
    lenlon=lonmax-1
    lenyear=2100-1850+1
    miss_val = -9.99e08

    lats = data2['lat'].values
    lons = data2['lon'].values
    weights = np.cos(np.deg2rad(lats))
    weights /= np.sum(weights)
    weights_reshaped = np.reshape(weights, (1, len(lats), 1))
    weights_reshaped = np.repeat(weights_reshaped, len(lons), axis=2)


# Specify the path to your NetCDF files
    file_pattern = "/scratch/cimes/hh9736/CMIP6_pr/"+ model_names[model] +"/pr*_r1i1p1f1_*.nc"

    # Get a list of file paths matching the pattern
    file_paths = glob.glob(file_pattern)

    # Open and concatenate the files one by one
    datasets = []
    for file_path in file_paths:
        dataset = xr.open_dataset(file_path)
        datasets.append(dataset)

    # Concatenate the datasets along the time dimension
    data = xr.concat(datasets, dim="time")
    data = data.sortby('time')
    annual_mean = data.groupby('time.year').mean(dim='time')


    # Select the first 100 years of data
    start_year = annual_mean.year[0]
    end_year = start_year + 99
    YEARP = annual_mean.sel(year=slice(start_year, end_year))
    mask=np.array(data1['mrsos'])
    
    mask=mask[0,:,:]
    mask[mask>10000]=1.e+20
    mask[mask<0.000000000000000000000001]=1.e+20
   
    weights_reshaped=weights_reshaped[0,:,:]
    
    mat1=YEARP['pr']
    for yy in range(0,100):
        pts=np.array(mat1[yy,:,:])

        la=spa_lag(pts, kn)
        pts[expolar, :] = np.nan

        pts_land=copy(pts)
        pts_land[mask>10000]=np.nan
        GINIland[0,yy]=sgini(pts_land[~np.isnan(pts_land)], weights_reshaped[~np.isnan(pts_land)])
        GINIsland[0,yy]=sgini_la(pts_land[~np.isnan(pts_land)],la[~np.isnan(pts_land)], weights_reshaped[~np.isnan(pts_land)])
        
        pts_ocean=copy(pts)
        pts_ocean[mask<10000]=np.nan
        GINIocean[0,yy]=sgini(pts_ocean[~np.isnan(pts_ocean)], weights_reshaped[~np.isnan(pts_ocean)])
        GINIsocean[0,yy]=sgini_la(pts_ocean[~np.isnan(pts_ocean)],la[~np.isnan(pts_ocean)], weights_reshaped[~np.isnan(pts_ocean)])
        
        pts_glob=copy(pts)
        GINIglob[0,yy]=sgini(pts_glob[~np.isnan(pts_glob)], weights_reshaped[~np.isnan(pts_glob)])
        GINIsglob[0,yy]=sgini_la(pts_glob[~np.isnan(pts_glob)],la[~np.isnan(pts_glob)], weights_reshaped[~np.isnan(pts_glob)])
        
        
    np.save('/scratch/cimes/hh9736/CMIP6_pr/GINI_spa_'+ str(latnouse) +''+ str(latuse) +'landP_' + model_names[model] + '.npy', np.array([GINIland]))
    np.save('/scratch/cimes/hh9736/CMIP6_pr/GINIs_spa_'+ str(latnouse) +''+ str(latuse) +'landP_' + model_names[model] + '.npy', np.array([GINIsland]))
    
    np.save('/scratch/cimes/hh9736/CMIP6_pr/GINI_spa_'+ str(latnouse) +''+ str(latuse) +'oceanP_' + model_names[model] + '.npy', np.array([GINIocean]))
    np.save('/scratch/cimes/hh9736/CMIP6_pr/GINIs_spa_'+ str(latnouse) +''+ str(latuse) +'oceanP_' + model_names[model] + '.npy', np.array([GINIsocean]))
    
    np.save('/scratch/cimes/hh9736/CMIP6_pr/GINI_spa_'+ str(latnouse) +''+ str(latuse) +'globP_' + model_names[model] + '.npy', np.array([GINIglob]))
    np.save('/scratch/cimes/hh9736/CMIP6_pr/GINIs_spa_'+ str(latnouse) +''+ str(latuse) +'globP_' + model_names[model] + '.npy', np.array([GINIsglob]))



In [ ]:
latuse = 90

for model in range(0,10):
    GINIland=np.array([np.full((lenyear),miss_val)])
    GINIsland=np.array([np.full((lenyear),miss_val)])

    GINIocean=np.array([np.full((lenyear),miss_val)])
    GINIsocean=np.array([np.full((lenyear),miss_val)])

    GINIglob=np.array([np.full((lenyear),miss_val)])
    GINIsglob=np.array([np.full((lenyear),miss_val)])
  
    MoranI=np.array([np.full((lenyear),miss_val)])
    
    mat1 = nc.Dataset('/scratch/cimes/hh9736/CMIP6_pr/' + model_names[model] + GRID[model])
    data1 = nc.Dataset('/scratch/cimes/hh9736/CMIP6_pr/' + model_names[model] + MASK[model])
    data2 = xr.open_dataset('/scratch/cimes/hh9736/CMIP6_pr/' + model_names[model] + MASK[model])
    lats = data2['lat'].values
    lats = lats.astype(int)
    expolar = np.logical_or(lats > latuse, lats < -latuse)
    
    kn=round(lats.shape[0]/18)
    
    latmax=np.size(mat1['pr'][1,:,1])+1
    lonmax=np.size(mat1['pr'][1,1,:])+1
    lenlat=latmax-1
    lenlon=lonmax-1
    lenyear=2100-1850+1
    miss_val = -9.99e08

    lats = data2['lat'].values
    lons = data2['lon'].values
    weights = np.cos(np.deg2rad(lats))
    weights /= np.sum(weights)
    weights_reshaped = np.reshape(weights, (1, len(lats), 1))
    weights_reshaped = np.repeat(weights_reshaped, len(lons), axis=2)


# Specify the path to your NetCDF files
    file_pattern = "/scratch/cimes/hh9736/CMIP6_pr/"+ model_names[model] +"/pr*_r1i1p1f1_*.nc"

    # Get a list of file paths matching the pattern
    file_paths = glob.glob(file_pattern)

    # Open and concatenate the files one by one
    datasets = []
    for file_path in file_paths:
        dataset = xr.open_dataset(file_path)
        datasets.append(dataset)

    # Concatenate the datasets along the time dimension
    data = xr.concat(datasets, dim="time")
    data = data.sortby('time')
    annual_mean = data.groupby('time.year').mean(dim='time')

    # Select the first 100 years of data
    start_year = annual_mean.year[0]
    end_year = start_year + 99
    YEARP = annual_mean.sel(year=slice(start_year, end_year))
    mask=np.array(data1['mrsos'])
    
    mask=mask[0,:,:]
    mask[mask>10000]=1.e+20
    mask[mask<0.000000000000000000000001]=1.e+20
   
    weights_reshaped=weights_reshaped[0,:,:]
    
    mat1=YEARP['pr']
    for yy in range(0,100):
        pts=np.array(mat1[yy,:,:])
  
        la=spa_lag(pts, kn)
        pts[expolar, :] = np.nan
        
        pts_land=copy(pts)
        pts_land[mask>10000]=np.nan
        GINIland[0,yy]=sgini(pts_land[~np.isnan(pts_land)], weights_reshaped[~np.isnan(pts_land)])
        GINIsland[0,yy]=sgini_la(pts_land[~np.isnan(pts_land)],la[~np.isnan(pts_land)], weights_reshaped[~np.isnan(pts_land)])
        
        pts_ocean=copy(pts)
        pts_ocean[mask<10000]=np.nan
        GINIocean[0,yy]=sgini(pts_ocean[~np.isnan(pts_ocean)], weights_reshaped[~np.isnan(pts_ocean)])
        GINIsocean[0,yy]=sgini_la(pts_ocean[~np.isnan(pts_ocean)],la[~np.isnan(pts_ocean)], weights_reshaped[~np.isnan(pts_ocean)])
        
        pts_glob=copy(pts)
        GINIglob[0,yy]=sgini(pts_glob[~np.isnan(pts_glob)], weights_reshaped[~np.isnan(pts_glob)])
        GINIsglob[0,yy]=sgini_la(pts_glob[~np.isnan(pts_glob)],la[~np.isnan(pts_glob)], weights_reshaped[~np.isnan(pts_glob)])
        
        
    np.save('/scratch/cimes/hh9736/CMIP6_pr/GINI_spa_'+ str(latuse) +'landP_' + model_names[model] + '.npy', np.array([GINIland]))
    np.save('/scratch/cimes/hh9736/CMIP6_pr/GINIs_spa_'+ str(latuse) +'landP_' + model_names[model] + '.npy', np.array([GINIsland]))
    
    np.save('/scratch/cimes/hh9736/CMIP6_pr/GINI_spa_'+ str(latuse) +'oceanP_' + model_names[model] + '.npy', np.array([GINIocean]))
    np.save('/scratch/cimes/hh9736/CMIP6_pr/GINIs_spa_'+ str(latuse) +'oceanP_' + model_names[model] + '.npy', np.array([GINIsocean]))
    
    np.save('/scratch/cimes/hh9736/CMIP6_pr/GINI_spa_'+ str(latuse) +'globP_' + model_names[model] + '.npy', np.array([GINIglob]))
    np.save('/scratch/cimes/hh9736/CMIP6_pr/GINIs_spa_'+ str(latuse) +'globP_' + model_names[model] + '.npy', np.array([GINIsglob]))
